Python Threat Hunting Tools: Part 3 — Interacting with APIs

An API, or Application Programming Interface, is a set of protocols, routines, and tools for building software applications. It defines a set of rules that allow software applications to interact with each other to exchange data and services. They can be considered a bridge between two applications that allows them to communicate with each other in a standard way.
The most common APIs you are likely to interact with are web APIs, which allow you to interact with a web-based application in a programmable manner. APIs can be public (which anyone can use) or private (which are only accessible by authorized parties). Again, most of the time you will use APIs that will require some form of authorization (username & password, token, etc.) as they will relate to an individual account or tenant and be private.

We will use Maltiverse free version API and Python 'requests' module to check malicious IP addresses and domains.

In [ ]:
# Malicious IP checking Method 1 - Using Maltiverse Python Module developed by Maltiverse team

#!pip3 install maltiverse
import ipaddress
from maltiverse import Maltiverse
api = Maltiverse()      # A paied version of API looks like api = Maltiverse(auth_token="..."), no parameters for the new created object api means this is a free version

while True:
    # Get input from user
    ip = input("Please enter IP to search for (or type exit, e, quit, q to quit): ")
    
    clean_ip = ip.strip()       # clean user input
    
    # Validate the given IP
    try:
        ipaddress.ip_address(clean_ip)
    except ValueError:
        print(f" - {clean_ip} is not a valid IP address. Please try again...\n")
        continue
    
    # query Maltiverse API for the IP
    result = api.ip_get(clean_ip)

    # check if the 'classification' key is present
    try:
        print(f"\n=> The IP address {clean_ip} has been identified as {result['classification']} by Maltiverse\n")
    except KeyError:
        print(f"\n - The IP address {clean_ip} cannot be classified by Maltiverse\n")
    
    if clean_ip == "quit" or "q" or "exit" or "e":
        print("Goodbye!")
        break

# Example malicious IPs:
# 104.223.120.159
# 117.184.105.34
# 101.36.121.4
# 110.189.222.98


=> The IP address 110.189.222.98 has been identified as suspicious by Maltiverse

Goodbye!


In [ ]:
# Malicious IP checking Method 2 - Using Python 'requests' Module

import ipaddress
import json
import requests

url = 'https://api.maltiverse.com/ip/'

while True:
    # Get input from user
    ip = input("Please enter IP to search for (or type exit, e, quit, q to quit): ")
    
    clean_ip = ip.strip()       # clean user input
    
    # Validate the given IP
    try:
        ipaddress.ip_address(clean_ip)
    except ValueError:
        print(f" - {clean_ip} is not a valid IP address. Please try again...\n")
        continue
    
    # query to Maltiverse API for a response (json format)
    response = requests.get(url + ip)
    # then, parse the json format object to get the relevant text data
    result = json.loads(response.text)

    # check if the 'classification' key is present
    try:
        print(f"\n=> The IP address {clean_ip} has been identified as {result['classification']} by Maltiverse\n")
    except KeyError:
        print(f"\n - The IP address {clean_ip} cannot be classified by Maltiverse\n")
    
    if clean_ip == "quit" or "q" or "exit" or "e":
        print("Goodbye!")
        break

# Example malicious IPs:
# 104.223.120.159
# 117.184.105.34
# 101.36.121.4
# 110.189.222.98


=> The IP address 104.223.120.159 has been identified as malicious by Maltiverse

Goodbye!


In [1]:
# Advanced continuous malicious IP and domain checker
import ipaddress
import json
import requests
import re

# Maltiverse API endpoints
ip_url = 'https://api.maltiverse.com/ip/'
domain_url = 'https://api.maltiverse.com/hostname/'

def is_valid_ip(ip_string):
    try:
        ipaddress.ip_address(ip_string)
        return True
    except ValueError:
        return False

def is_valid_domain(domain_string):
    domain_pattern = r'^(?:[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,}$'
    return re.match(domain_pattern, domain_string) is not None

def check_maltiverse(input_value, input_type):
    try:
        if input_type == 'ip':
            response = requests.get(ip_url + input_value, timeout=10)
        else:
            response = requests.get(domain_url + input_value, timeout=10)
        
        if response.status_code == 200:
            result = json.loads(response.text)
            return result
        elif response.status_code == 404:
            return None  # Not found in Maltiverse database
        else:
            print(f" - API request failed with status code: {response.status_code}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f" - Error connecting to Maltiverse API: {e}")
        return None
    except json.JSONDecodeError:
        print(f" - Error parsing API response")
        return None

def main():
    print("Maltiverse IP and Domain Checker")
    print("=" * 40)
    
    while True:
        # Get input from user
        user_input = input("\nPlease enter IP address or domain to check (or type 'exit', 'e', 'quit', 'q' to quit): ")
        
        clean_input = user_input.strip().lower()
        
        # Check for exit conditions
        if clean_input in ['quit', 'q', 'exit', 'e']:
            print("Goodbye!")
            break
        
        original_input = user_input.strip()
        
        # Determine if input is IP or domain
        if is_valid_ip(original_input):
            input_type = 'ip'
            print(f"\n=> Checking IP address: {original_input}")
        elif is_valid_domain(original_input):
            input_type = 'domain'
            print(f"\n=> Checking domain: {original_input}")
        else:
            print(f" - '{original_input}' is not a valid IP address or domain name. Please try again...")
            continue
        
        # Query Maltiverse API
        result = check_maltiverse(original_input, input_type)
        
        if result is None:
            print(f" - The {input_type} '{original_input}' cannot be classified by Maltiverse (not found in database)")
        else:
            # Check if the 'classification' key is present
            if 'classification' in result:
                classification = result['classification']
                print(f" - The {input_type} '{original_input}' has been identified as '{classification}' by Maltiverse")
                
                # Additional information if available
                if 'tag' in result and result['tag']:
                    tags = ', '.join(result['tag'])
                    print(f" - Tags: {tags}")
                    
                if 'blacklist' in result and result['blacklist']:
                    blacklists = []
                    for bl in result['blacklist']:
                        if 'description' in bl:
                            blacklists.append(bl['description'])
                    if blacklists:
                        print(f" - Found in blacklists: {', '.join(blacklists)}")
            else:
                print(f" - The {input_type} '{original_input}' has no classification information available")

if __name__ == "__main__":
    main()

# Example malicious IPs:
# 104.223.120.159
# 117.184.105.34
# 101.36.121.4
# 110.189.222.98

# Example domains to test:
# google.com
# ameteksen.com
# assso.net
# caref1rst.com

Maltiverse IP and Domain Checker

=> Checking domain: baidu.com
 - The domain 'baidu.com' has been identified as 'whitelist' by Maltiverse
 - Tags: cobaltstrike, cs-watermark-426352781, tencent-net-ap shenzhen tencent computer systems company limited, agentemis, beacon, cobeacon
 - Found in blacklists: Cobalt Strike, Malicious URL, Top 100K Site, Top Popularity Site
 - '' is not a valid IP address or domain name. Please try again...

=> Checking domain: assso.net
 - The domain 'assso.net' has been identified as 'malicious' by Maltiverse
 - Found in blacklists: Malicious Domain

=> Checking domain: caref1rst.com
 - The domain 'caref1rst.com' has been identified as 'malicious' by Maltiverse
 - Found in blacklists: Malicious Domain

=> Checking domain: google.com
 - The domain 'google.com' has been identified as 'whitelist' by Maltiverse
 - Tags: azorult, c2, historicalandnew, puffstealer, rultazo, blacknet, lokibot, burkina, loki, lokipws, cobaltstrike, cs-watermark-100000, webnx inc., a